# Traverse: Records Community Detection

Build a genre/style co-occurrence graph from a records CSV, detect
communities via Louvain, and visualize in Cosmograph with community
clustering.  Clicking a node shows associated album/record titles.

**No timeline** — this notebook focuses on structure, not chronology.

**Prerequisites:**
```bash
pip install -e ".[dev]"
cd src/traverse/cosmograph/app && npm install && npm run build
```

## 1. Configuration

In [1]:
from pathlib import Path

RECORDS_CSV = Path(r"C:\Users\xtrem\Documents\Datasets\records.csv")
OUT_DIR = Path("_out")
FORCE = False  # set True to rebuild cache

## 2. Build or Load Graph

Build the co-occurrence graph from the records CSV (or load from cache).
The first run streams the full CSV (~30 min); subsequent runs load instantly.
Set `FORCE = True` above to rebuild from scratch.

In [ ]:
from traverse.graph.genres_graph import build_genre_graph
from traverse.graph.cache import GraphCache

cache = GraphCache(
    cache_dir=OUT_DIR,
    build_fn=lambda: build_genre_graph(
        RECORDS_CSV,
        min_cooccurrence=2,
        max_nodes=5_000,
        max_edges=150_000,
    ),
    force=FORCE,
)
graph, records_df = cache.load_or_build()
print(f"Graph: {len(graph['points'])} nodes, {len(graph['links'])} edges")
print(f"Records: {len(records_df):,} rows")

Loading graph from cache…


Graph: 708 nodes, 86021 edges
Records: 14,609,074 rows


  708 nodes, 86021 edges, 14,609,074 records


## 3. Community Detection

Run Louvain community detection and add community labels to each node.

In [3]:
from collections import Counter
from traverse.graph.community import add_communities, CommunityAlgorithm

graph = add_communities(graph, CommunityAlgorithm.LOUVAIN, seed=42)

comm_counts = Counter(pt["community"] for pt in graph["points"])
print(f"{len(comm_counts)} communities:")
for comm_id, count in comm_counts.most_common():
    print(f"  Community {comm_id}: {count} nodes")

6 communities:
  Community 0: 158 nodes
  Community 1: 145 nodes
  Community 2: 138 nodes
  Community 3: 130 nodes
  Community 4: 73 nodes
  Community 5: 64 nodes


## 4. Export & Serve

Export the community graph JSON and start the Cosmograph server.

In [ ]:
from traverse.graph.adapters_cosmograph import CosmographAdapter
from traverse.cosmograph.server import serve, _default_dist_dir

# Export graph JSON with community as default cluster
meta = {"clusterField": "community"}
out_path = _default_dist_dir() / "cosmo_records_community.json"
CosmographAdapter.write(graph, out_path, meta=meta)
print(f"Wrote {out_path} ({len(graph['points'])} nodes, {len(graph['links'])} edges)")
print()
print("Starting server — open in browser:")
print("  http://127.0.0.1:8080/?data=/cosmo_records_community.json")
print()
print("Press Ctrl+C (or interrupt the kernel) to stop.")

serve(port=8080)

Wrote C:\Users\xtrem\Documents\Projects\GitHub\traverse_vc\src\traverse\cosmograph\app\dist\cosmo_records_community.json (708 nodes, 86021 edges)

Graph has 86,729 items — writing compact JSON (override with indent=None)
Wrote C:\Users\xtrem\Documents\Projects\GitHub\traverse_vc\src\traverse\cosmograph\app\dist\cosmo_records_community.json (5.1 MB)




Starting server — open in browser:
  http://127.0.0.1:8080/?data=/cosmo_records_community.json

Press Ctrl+C (or interrupt the kernel) to stop.
Serving C:\Users\xtrem\Documents\Projects\GitHub\traverse_vc\src\traverse\cosmograph\app\dist at http://127.0.0.1:8080
